# Data Quality 

Dopo aver completato la fase di Data Ingestion tramite lo scraping il secondo step è quello di valutare la qualità dei dati raccolti.  
Il dataframe studiato presenta 18 colonne. Ogni colonna presenta delle differenze dalle altre e ci si aspetta determinati valori da queste.

Dopo aver settato:
1. la datasource
2. suite e le relative expectations (si rimanda al noteook SuiteSetUp_main)
3. creazione del checkpoint e validazione

Si condurrà una validazione del sample

In [ ]:
import great_expectations as gx
from ruamel import yaml
from great_expectations.data_context.types.resource_identifiers import ExpectationSuiteIdentifier
import os
import sys
sys.path.insert(0, '../funzioni')
from funzioni import *

In [ ]:
context = gx.get_context()

### Configurazione della datasource 

In [ ]:
datasource_config: dict = {
    "name": "glassdoor_scraping",
    "class_name": "Datasource",
    "module_name": "great_expectations.datasource",
    "execution_engine": {
        "class_name": "PandasExecutionEngine",
        "module_name": "great_expectations.execution_engine",
    },
     "data_connectors": {
        "all": {
            "class_name": "InferredAssetFilesystemDataConnector",
            "base_directory": "../data",
            "default_regex": {"pattern": "(.*)\\.csv",
            "group_names": ["data_asset_name"]},
            #"batch_spec_passthrough": {
            #    "reader_method": "read_csv",
            #    "reader_options": {
            #        "header": True,
            #        "inferSchema": True,
            #    },
            #},
        }
    },
}

In [ ]:
#check configurazione
context.test_yaml_config(yaml.dump(datasource_config))

In [ ]:
## try to add
try:
    context.get_datasource(datasource_config["name"])
except ValueError:
    context.add_datasource(**datasource_config)
else:
    print(
        f"The datasource {datasource_config['name']} already exists in your Data Context!"
    )

### Indicazione delle suite

Si rimanda al noteook SuiteSetUp_main per eventuali modifiche della stessa

In [ ]:
a = context.list_expectation_suite_names()
suite_identifier = ExpectationSuiteIdentifier(expectation_suite_name=a[0])
context.build_data_docs(resource_identifiers=[suite_identifier])
context.open_data_docs(resource_identifier=suite_identifier)
print('http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/expectations/'+ a[0] +'.html')

### Creazione del checkpoint e validazione

E' stata creata una funzione che prendendo come input il data-asset (sotto gruppo della data-source) crerà per ogni data-asset un checkpoint.
Poiché ogni data-asset corrisponde al giorno in cui è stato lanciato il processo di data-ingestion verrà creato un checkpoint per ogni giorno in cui appunto ha girato il processo di data-ingestion

Per le funzioni si rimanda alla percorso `../funzioni/funzioni.py`.
Per ogni validazione del checkpoint saranno disponibili i risultati in formato HTML

In [ ]:
run_dataq()